In [ ]:
from fenics import *
from fenics_adjoint import *
import matplotlib.pyplot as plt

N = 20
alpha = 1.0
beta = 1.0
c1 = 0
c2 = 1


iteration_count = 0
iteration_plots = [1,2,3,4,5, 10, 20, 25, 30, 40, 50, 80, 100, 110, 120, 150, 200, 250, 300]

def setup(N):
    mesh = Mesh(UnitIntervalMesh(N))
    V = FunctionSpace(mesh, "CG", 1)
    f = Function(V)
    dS = Measure("dS", domain=mesh)
    bm = MeshFunction("size_t", mesh, 0)
    bm.array()[0] = 1
    bm.array()[-1] = 2
    ds = Measure("ds", domain=mesh, subdomain_data=bm)
    n = FacetNormal(mesh)
    J1 = assemble(((f - c1)**2*ds(1) + (f - c2)**2*ds(2)))
    J2 = assemble(jump(grad(f), n)**2*dS)
    #J2 = assemble((inner(grad(f), grad(f)))*dx)
    J = alpha*J1 + beta*J2
    return J, f
    
def derivative_cb(j, dj, m):
    global iteration_count
    if iteration_count in iteration_plots:
        plt.figure()
        plot(m, "-.")
        plt.title(f"iteration {iteration_count}")
        plt.grid()
    iteration_count += 1
    
J,f = setup(N)

reduced_functional = ReducedFunctional(J, Control(f), derivative_cb_post = derivative_cb)

f_opt = minimize(reduced_functional, method = 'BFGS', options = {'disp': True})


In [ ]:
print_optimization_methods()

In [ ]:
for N in [10, 50]:
    print("==========================================")
    print(f"            N = {N}                      ")
    print("==========================================")
    for method in ["BFGS", "CG", "Newton-CG","SLSQP" ]:
        print(f"Using {method}:")
        J,f = setup(N)
        reduced_functional = ReducedFunctional(J, Control(f), derivative_cb_post = derivative_cb)
        f_opt = minimize(reduced_functional, method = method, options = {'disp': True})


In [ ]:
minimize?